In [64]:
import numpy as np
import pandas as pd
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=64
EVALUATE_EVERY=15
FILTER_SIZES=5
L2_REG_LAMBDA=0.001
LOG_DEVICE_PLACEMENT=False
NUM_EPOCHS=20
NUM_FILTERS=256
STOP_WRODS=False
fname = "w2vmodel" #w2vmodel128

In [36]:
def washstr(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [37]:
import nltk
import string
import re

def tokenizer(text):
    lower_txt = washstr(text)
    tokens = lower_txt.split(" ")
    if not STOP_WRODS:
        return tokens
    nonstop = []
    for token in tokens:
        if token not in stopwords:
            nonstop.append(token)
    return nonstop

In [38]:
def extfeat(X_train):
    neg = pd.read_csv('negative-words.txt')['neg'].values
    pos = pd.read_csv('positive-words.txt')['pos'].values
    neg = set(neg)
    pos = set(pos)
    
    ext = []
    for i in range(len(X_train)):
        p = 0.0
        n = 0.0
        total = 0.0
        for j in X_train[i]:
            if j != " ":
                total += 1
                if j in pos:
                    p += 1
                if j in neg:
                    n += 1
        if n == 0:
            ratio = p
        else:
            ratio = p/n
        ext.append([p,n,p/total,n/total,ratio])
    return ext

In [39]:
import gensim
def word2vec(X_train):
    X = []
    for i in range(X_train.shape[0]):
        X.append(tokenizer(X_train[i]))
    
    for i in range(len(X)):
        while len(X[i]) < max_len:
            X[i].append(" ")
        
    model = gensim.models.Word2Vec(X, min_count=1,size=EMBEDDING_DIM)
    return model

In [40]:
def getwvec(X_train,model):
    x = []
    for i in range(len(X_train)):
        x.append(model.wv[X_train[i]])
    return np.array(x)

In [41]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    print(data_size)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [42]:
def train_step(x_batch, y_batch):
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: DROPOUT_KEEP_PROB
    }
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

def dev_step(x_batch, y_batch):
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: 1.0
    }
    step, summaries, loss, accuracy = sess.run(
        [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

In [43]:
train_set = pd.read_csv('labeledmusic.csv')
stop_words = pd.read_csv('stopwords.txt')
stopwords = set(stop_words['stopwords'].values)

In [44]:
print(stopwords)

{'me', 'should', 'here', 'we', 'herself', 'most', 'between', 'themselves', 'how', 'the', 'are', 't', 'having', 'with', 'my', 'why', 'and', 'because', 'your', 'until', 'then', 'any', 'them', 'doing', 'i', 'more', 'yourselves', 'just', 'few', 'been', 'they', 'out', 'have', 'to', 'but', 'during', 'has', 'both', 'of', 'not', 'on', 'ourselves', 'am', 'a', 'what', 'other', 'nor', 'hers', 'while', 'had', 'do', 'yourself', 'myself', 'all', 'each', 'than', 'for', 'those', 'will', 'such', 'against', 'over', 'under', 'can', 'their', 'some', 'don', 'him', 'she', 'be', 'when', 'who', 'these', 'it', 'above', 'that', 'again', 'too', 'through', 'were', 'off', 'itself', 'in', 'ours', 'up', 'there', 'down', 'this', 'does', 'only', 'so', 'his', 'its', 'or', 'same', 'now', 's', 'further', 'at', 'did', 'below', 'very', 'own', 'her', 'as', 'an', 'theirs', 'he', 'whom', 'if', 'which', 'you', 'no', 'after', 'is', 'once', 'into', 'where', 'was', 'himself', 'by', 'from', 'yours', 'our', 'being', 'about', 'befor

In [45]:
lytrain = train_set['text'].values 

y_train = train_set['mood'].values

In [46]:
print(lytrain.shape, y_train.shape)

(1199,) (1199,)


In [47]:
X_train = []
Y = []

max_len = 0

for i in range(lytrain.shape[0]):
    X_train.append(tokenizer(lytrain[i]))
    max_len = max(max_len, len(X_train[i]))

    if y_train[i] == "sad":
        Y.append([0,1])
    else:
        Y.append([1,0])

In [48]:
print(len(X_train), max_len)

1199 1266


In [49]:
# fill each lyric, to make it of 1087 tokens
for i in range(len(X_train)):
    while len(X_train[i]) < max_len:
        X_train[i].append(" ")

In [50]:
for i in range(len(X_train)):
    if len(X_train[i]) != max_len:
        print("error")

In [51]:
print(lytrain.shape)

(1199,)


In [18]:
# # since the dataset is too small, word vector may not be good, so use a greater lyric set to generate w2v model
# unlabeled = pd.read_csv('songdata.csv')
# w2vtran = unlabeled['text'].values
# w2vtran = np.row_stack((w2vtran.reshape(-1,1),lytrain.reshape(-1,1)))

In [19]:
# model = word2vec(w2vtran[:,0])
# model.save(fname)

In [20]:
model = gensim.models.Word2Vec.load(fname)

In [21]:
# print(w2vtran.shape)

In [52]:
x_ = getwvec(X_train,model)
print(x_.shape)

(1199, 1266, 64)


In [53]:
# extra = extfeat(X_train)

In [54]:
# extra = np.array(extra)
# print(extra.shape)

In [55]:
# x_ = x_.reshape(-1,x_.shape[1]*x_.shape[2])

In [56]:
# x = np.column_stack((x_,extra))

In [57]:
# from sklearn.decomposition import PCA 
# pca=PCA(n_components=0.998)
# x = pca.fit_transform(x)

In [58]:
x = x_
y = np.array(Y)

In [59]:
print(x.shape, y.shape)

(1199, 1266, 64) (1199, 2)


In [60]:
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
dev_sample_index = -1 * int(DEV_SAMPLE_PERCENTAGE * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

del x, y, x_shuffled, y_shuffled

print("yTrain/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
print("xTrain/Dev split: {:d}/{:d}".format(len(x_train), len(x_dev)))

yTrain/Dev split: 1080/119
xTrain/Dev split: 1080/119


In [ ]:
#! /usr/bin/env python

import tensorflow as tf
import numpy as np
import time
import datetime
from cnn_model import CNN

# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=ALLOW_SOFT_PLACEMENT,
      log_device_placement=LOG_DEVICE_PLACEMENT)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            embedding_dim=EMBEDDING_DIM,
            filter_size=FILTER_SIZES,
            num_filters=NUM_FILTERS,
            l2_reg_lambda=L2_REG_LAMBDA)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])

        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), BATCH_SIZE, NUM_EPOCHS)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            print(len(x_batch))
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % EVALUATE_EVERY == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev)
                print("")


INFO:tensorflow:Summary name conv-maxpool-5/W:0/grad/hist is illegal; using conv-maxpool-5/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-5/W:0/grad/sparsity is illegal; using conv-maxpool-5/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-5/b:0/grad/hist is illegal; using conv-maxpool-5/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-5/b:0/grad/sparsity is illegal; using conv-maxpool-5/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name W:0/grad/hist is illegal; using W_0/grad/hist instead.
INFO:tensorflow:Summary name W:0/grad/sparsity is illegal; using W_0/grad/sparsity instead.
INFO:tensorflow:Summary name output/b:0/grad/hist is illegal; using output/b_0/grad/hist instead.
INFO:tensorflow:Summary name output/b:0/grad/sparsity is illegal; using output/b_0/grad/sparsity instead.
1080
64
2017-12-19T22:58:53.558926: step 1, loss 5.76553, acc 0.5625
64
2017-12-19T22:58:55.033590: step 2, loss 10.4408, acc 0.421875
64
2017-12-19

2017-12-19T23:01:35.867375: step 105, loss 1.33784, acc 0.613445

64
2017-12-19T23:01:37.276698: step 106, loss 3.19506, acc 0.640625
64
2017-12-19T23:01:38.869206: step 107, loss 4.21748, acc 0.5625
64
2017-12-19T23:01:40.279010: step 108, loss 3.38686, acc 0.65625
64
2017-12-19T23:01:41.709891: step 109, loss 2.66365, acc 0.609375
64
2017-12-19T23:01:43.124098: step 110, loss 2.3393, acc 0.75
64
2017-12-19T23:01:44.525042: step 111, loss 3.17306, acc 0.71875
64
2017-12-19T23:01:45.929296: step 112, loss 1.85566, acc 0.6875
64
2017-12-19T23:01:47.339785: step 113, loss 3.49839, acc 0.640625
64
2017-12-19T23:01:48.797390: step 114, loss 2.89247, acc 0.640625
64
2017-12-19T23:01:50.215235: step 115, loss 2.71748, acc 0.609375
64
2017-12-19T23:01:51.626302: step 116, loss 2.1478, acc 0.75
64
2017-12-19T23:01:53.027740: step 117, loss 4.13979, acc 0.5
64
2017-12-19T23:01:54.492530: step 118, loss 3.28543, acc 0.59375
56
2017-12-19T23:01:56.110799: step 119, loss 3.37642, acc 0.517857
64
2

2017-12-19T23:04:26.214656: step 219, loss 2.02729, acc 0.65625
64
2017-12-19T23:04:27.622560: step 220, loss 1.52452, acc 0.71875
56
2017-12-19T23:04:28.854492: step 221, loss 1.73108, acc 0.660714
64
2017-12-19T23:04:30.314882: step 222, loss 1.57131, acc 0.734375
64
2017-12-19T23:04:32.324106: step 223, loss 2.48689, acc 0.59375
64
2017-12-19T23:04:34.258980: step 224, loss 1.77456, acc 0.703125
64
2017-12-19T23:04:36.294096: step 225, loss 1.59048, acc 0.671875

Evaluation:
2017-12-19T23:04:37.523567: step 225, loss 0.926494, acc 0.630252

64
2017-12-19T23:04:39.173638: step 226, loss 2.24946, acc 0.578125
64
2017-12-19T23:04:41.121698: step 227, loss 1.51531, acc 0.8125
64
2017-12-19T23:04:42.564916: step 228, loss 1.76839, acc 0.703125
64
2017-12-19T23:04:43.993390: step 229, loss 1.25184, acc 0.640625
64
2017-12-19T23:04:45.421056: step 230, loss 0.802777, acc 0.8125
64
2017-12-19T23:04:46.843680: step 231, loss 1.70929, acc 0.6875
64
2017-12-19T23:04:48.254812: step 232, loss 1